<a href="https://colab.research.google.com/github/amandacsgomes/TrilhaDaAprovacao/blob/main/Trilha_da_Aprova%C3%A7%C3%A3o_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

O projeto **Trilha da Aprovação** desenvolve um agente de IA capaz de auxiliar concurseiros em sua jornada de estudos. Suas funcionalidades primárias incluem a geração automática de um plano de estudo customizado, baseado na disponibilidade do estudante e no conteúdo do edital do concurso, além disso inclui um chat que facilita a consulta de informações do edital, para que os usuários possam acessar informações específicas do edital de maneira ágil.

In [ ]:
!pip -q install google-genai
!pip -q install PyMuPDF
!pip -q install google-adk

In [ ]:
import os
import ast
import json
import fitz  # PyMuPDF
from google.colab import userdata
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google import genai
from google.genai import types
from google.colab import files
from datetime import datetime, timedelta
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import warnings

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=os.environ['GOOGLE_API_KEY'])
model = 'gemini-2.0-flash'

warnings.filterwarnings("ignore")

In [ ]:
#Gemini gerou pra mim o código de extração de texto do PDF
def extract_text_from_pdf(pdf_path):
  try:
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(doc.page_count):
      page = doc.load_page(page_num)
      text += page.get_text()
    doc.close()
    return text
  except Exception as e:
    print(f"Error extracting text from PDF: {e}")
    return None

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
###############################################
# --- Agente 1: Buscador de informações --- #
###############################################
def agente_buscador(edital):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction=f"""
        Sempre que eu pedir uma data, considera o formato dd/mm/aaaa.
        Me retorne somente a resposta da pergunta, sem comentários ou emojis.
        Considere o conteúdo do edital a seguir para responder todas as próximas perguntas: {edital}""",
        description="Agente que busca os dados dentro do texto do arquivo (já extraído do pdf)"
    )

    texto_cargos = call_agent(buscador,
                              """Faz uma única lista com os cargos do edital com a estrutura pra eu usar em dicionario do Pyhton:
                              [{indice: 1, emoji:"um emoji que combine com cargo A", descricao: "Cargo A", vagas: "quantidade de vagas do cargo A", salario: "salário do cargo A", nivel: "nível do cargo A"},
                               {indice: 2, emoji:"um emoji que combine com cargo B", descricao: "Cargo B", vagas: "quantidade de vagas do cargo B", salario: "salário do cargo B", nivel: "nível do cargo B"}]
                              Sem nenhum comentário ou caracter adicional. Apenas a parte entre colchetes[ ].
                              """)

    conteudo_limpo = texto_cargos.replace("```json", "").replace("```", "").strip()

    lista_cargos = json.loads(conteudo_limpo)

    data_prova = call_agent(buscador, f"Qual é a data da prova?")
    data_prova = data_prova.strip()
    conteudo_programatico = call_agent(buscador, f"Lista o contéudo programático desse edital")

    return data_prova, conteudo_programatico, lista_cargos

In [ ]:
#############################################
# --- Agente 2: Planejador de estudos --- #
#############################################
def agente_planejador(cargo, conteudo_programatico, qtd_dias, qtd_horas, data_prova, semanas):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        instruction="""Você é um assistente pessoal que vai ajudar um concurseiro,
                      você sempre responde de forma motivadora, não muito formal, de maneira direta e objetiva.
                      Sempre que eu pedir uma data, considera o formato dd/mm/aaaa.
                      Monta um plano de estudos para concurso, considerando o conteúdo programático indicado, o cargo escolhido e a disponibilidade informada.""",
        description="Agente que monta o plano de estudos com base no cargo escolhido e disponibilidade"
    )

    inicio_do_plano = (datetime.now() + timedelta(days=1)).date()
    entrada_do_agente_planejador = f"""Cargo:{cargo}\nConteúdo programático: {conteudo_programatico}\n
                                      Disponibilidade: {qtd_dias} por semana, {qtd_horas} por dia.\nData da prova: {data_prova}\n
                                      Início do plano: {inicio_do_plano}, duração: {semanas} semanas"""
    # Executa o agente
    plano_de_estudo = call_agent(planejador, entrada_do_agente_planejador)
    return plano_de_estudo

In [ ]:
def chat_edital(dados_edital, plano_de_estudos):
  chat_config = types.GenerateContentConfig(
    system_instruction=f"""Você é um assistente pessoal que vai ajudar um concurseiro,
                          você sempre responde de forma motivadora e não muito formal, de maneira direta e objetiva.
                          Considere o edital e o meu plano de estudo abaixo para responder a todas as perguntas.
                          Edital: {dados_edital}
                          Plano de estudos: {plano_de_estudos}""")

  chat = client.chats.create(model=model, config=chat_config)

  print("\n\n[Você pode digitar 'sair' a qualquer momento]")
  prompt = input("Tem mais alguma pergunta sobre esse edital?\n")

  if prompt == 'não':
    print("\nBons estudos!✅")
  else:
    if prompt == 'sim':
      prompt = input("Qual é a sua pergunta?\n")
    while prompt != 'sair':
      resposta = chat.send_message(prompt)
      print(resposta.text)
      prompt = input()
    print("\nBons estudos!✅")


In [ ]:
print("Olá!😀 Seja bem-vindo à Trilha da Aprovação! 📝✅")
print("Nosso objetivo é te ajudar a traçar um plano de estudos. 📆")
print("Você pode começar enviando o pdf do edital do concurso que vai fazer (atual ou anterior):\n")

uploaded = files.upload()

texto_pdf = extract_text_from_pdf(list(uploaded.keys())[0])

print("\nAnalisando...")

data_prova, conteudo_programatico, lista_cargos = agente_buscador(texto_pdf)

dias_ate_prova = (datetime.strptime(data_prova, "%d/%m/%Y") - datetime.now()).days

if dias_ate_prova <= 0:
    edital_atual = False
    print(f"\nEsse é um edital antigo, a prova foi em: {data_prova} ⌛")
else:
    edital_atual = True
    semanas = (datetime.strptime(data_prova, "%d/%m/%Y") - datetime.now()).days // 7
    if dias_ate_prova <= 10:
        print(f"\nTemos menos de 10 dias para a prova - {data_prova}😧")
    else:
        print(f"\nData da prova - {data_prova}📝\nSemanas: {semanas}")


print("Agora vamos montar seu plano de estudos.\nOs cargos disponíveis são:\n")
#print(dados_edital["cargos"])
for cargo in lista_cargos:
  print(f"{cargo['indice']}. {cargo['emoji']} {cargo['descricao']} ({cargo['nivel']}) - {cargo['vagas']} vaga(s) - Salário: {cargo['salario']}")

cargo = input("\nQual dos cargos acima você escolheu? ")
qtd_dias = input("Quantos dias por semana você vai estudar? ")
qtd_horas = input("Quantas horas por dia? ")
if not edital_atual:
    semanas = input("Por quantas semanas? ")

print("\nOk! Agora vamos montar seu plano de estudos.📆\nAguarde...")

cargo = int(cargo) - 1

plano_de_estudos = agente_planejador(lista_cargos[cargo]['descricao'], conteudo_programatico, qtd_dias, qtd_horas, data_prova, semanas)
display(to_markdown(plano_de_estudos))

chat_edital(texto_pdf, plano_de_estudos)

Olá!😀 Seja bem-vindo à Trilha da Aprovação! 📝✅
Nosso objetivo é te ajudar a traçar um plano de estudos. 📆
Você pode começar enviando o pdf do edital do concurso que vai fazer (atual ou anterior):



Saving 1731676739.pdf to 1731676739 (22).pdf

Analisando...

Esse é um edital antigo, a prova foi em: 13/10/2024 ⌛
Agora vamos montar seu plano de estudos.
Os cargos disponíveis são:

1. 🧑‍🏫 Auxiliar de Serviços Escolares (FUNDAMENTAL) - 242 vaga(s) - Salário: R$ 2.294,39
2. 🍎 Professor A (MÉDIO) - 182 vaga(s) - Salário: R$ 2.411,11
3. 📚 Professor A – AEE (Atendimento educacional especializado) (MÉDIO) - 31 vaga(s) - Salário: R$ 2.411,11
4. ✍️ Professor A – Trad. e Interprete Libras (MÉDIO) - 9 vaga(s) - Salário: R$ 2.411,11
5. 🗣️ Professor C – Língua Portuguesa (SUPERIOR) - 29 vaga(s) - Salário: R$ 2.906,82
6. ➗ Professor C – Matemática (SUPERIOR) - 33 vaga(s) - Salário: R$ 2.906,82
7. 🏛️ Professor C – História (SUPERIOR) - 13 vaga(s) - Salário: R$ 2.906,82
8. 🌎 Professor C – Geografia (SUPERIOR) - 22 vaga(s) - Salário: R$ 2.906,82
9. 🔬 Professor C – Ciências (SUPERIOR) - 3 vaga(s) - Salário: R$ 2.906,82
10. 🏃 Professor C – Ed. Física (SUPERIOR) - 30 vaga(s) - Salário: R$ 2.906,82
11.

> 🚀 Recebido! Com esse plano, sua aprovação está mais perto do que você imagina. Vamos estruturar seus estudos para o cargo de Auditor Municipal de Controle Interno com foco total no sucesso!
> 
> ### Plano de Estudos Turbinado (20/05/2024 a 08/06/2024)
> 
> **Visão Geral:**
> 
> *   **Carga Horária:** 6 dias por semana, 4 horas por dia.
> *   **Foco:** Cobrir todos os tópicos do edital de forma equilibrada.
> *   **Estratégia:** Revisão teórica + resolução de questões + simulados.
> 
> **Distribuição Semanal:**
> 
> *   **Dias da Semana:** Escolha 6 dias que funcionem melhor para você (ex: Seg-Sáb).
> *   **Horários:** Divida as 4 horas diárias em blocos de 50 minutos de estudo + 10 minutos de descanso.
> 
> **Conteúdo Programático Detalhado:**
> 
> **MÓDULO I – CONHECIMENTOS BÁSICOS**
> 
> *   **LÍNGUA PORTUGUESA**
>     *   Interpretação de Textos (2h semanais)
>     *   Gramática (2h semanais)
> *   **RACIOCÍNIO LÓGICO-MATEMÁTICO**
>     *   Lógica Proposicional (2h semanais)
>     *   Matemática Financeira (2h semanais)
> *   **AUDITORIA, CONTROLE E AVALIAÇÃO**
>     *   Normas de Auditoria (3h semanais)
>     *   Controle Interno (3h semanais)
>     *   Avaliação de Programas e Políticas Públicas (2h semanais)
> *   **LEGISLAÇÃO MUNICIPAL**
>     *   Lei Orgânica do Município (2h semanais)
>     *   Estatuto dos Servidores (2h semanais)
>     *   Legislação Tributária Municipal (2h semanais)
> 
> **MÓDULO II – CONHECIMENTOS ESPECÍFICOS**
> 
> *   **AUDITOR MUNICIPAL DE CONTROLE INTERNO**
>     *   Contabilidade Pública (4h semanais)
>     *   Administração Financeira e Orçamentária (4h semanais)
>     *   Direito Administrativo (4h semanais)
>     *   Direito Constitucional (4h semanais)
>     *   Responsabilidade Fiscal (Lei Complementar nº 101/2000) (4h semanais)
>     *   Controle da Administração Pública (4h semanais)
> 
> **Cronograma Semanal Sugerido:**
> 
> *   **Segunda-feira:**
>     *   Manhã: Língua Portuguesa (2h)
>     *   Tarde: Contabilidade Pública (2h)
> *   **Terça-feira:**
>     *   Manhã: Raciocínio Lógico-Matemático (2h)
>     *   Tarde: Administração Financeira e Orçamentária (2h)
> *   **Quarta-feira:**
>     *   Manhã: Auditoria, Controle e Avaliação (3h)
>     *   Tarde: Direito Administrativo (1h)
> *   **Quinta-feira:**
>     *   Manhã: Legislação Municipal (2h)
>     *   Tarde: Direito Constitucional (2h)
> *   **Sexta-feira:**
>     *   Manhã: Responsabilidade Fiscal (2h)
>     *   Tarde: Controle da Administração Pública (2h)
> *   **Sábado:**
>     *   Manhã: Revisão da Semana (2h)
>     *   Tarde: Resolução de Questões (2h)
> 
> **Estratégias de Estudo:**
> 
> *   **Teoria:** Use videoaulas, PDFs e livros.
> *   **Questões:** Resolva muitas questões da banca do seu concurso para pegar o jeito.
> *   **Revisão:** Revise a matéria toda semana para não esquecer nada.
> *   **Simulados:** Faça simulados para testar seus conhecimentos e se acostumar com o tempo de prova.
> *   **Descanso:** Não esqueça de descansar! O sono é fundamental para o aprendizado.
> 
> **Recursos Adicionais:**
> 
> *   **Aplicativos:** Utilize aplicativos de organização e planejamento de estudos.
> *   **Grupos de Estudo:** Participe de grupos para trocar ideias e tirar dúvidas.
> *   **Mentoria:** Considere ter um mentor para te guiar e motivar.
> 
> Com esse plano, você vai estar preparado para brilhar na prova de Auditor Municipal de Controle Interno! Acredite em você e siga firme! 😉
> 




[Você pode digitar 'sair' a qualquer momento]
Tem mais alguma pergunta sobre esse edital?
não

Bons estudos!✅
